In [6]:
# all the helper functions
from ctypes import c_uint16
def check_dual(a:list, b1:list, b2:list):
    # b1 -> even, b2 -> odd
    assert(len(a)==(len(b1)+len(b2)))
    print("[+] checking...")
    for i in range(len(b1)):
        if(a[i*2]  != b1[i] or a[i*2+1] != b2[i]):
            print(i,"a: ",a[i*2],a[i*2+1], "b: ", b1[i],b2[i])

def to_hex(a:list) -> list:
    r = []
    v = c_uint16()
    for i in a:
        v.value = i
        r.append(hex(v.value))
    return r


In [ ]:
# checks e1 in ACCU1
# e1 = HASH -> CBD -> ACCU1
# simulation time: 36395ps ~ 77695ps when ACCU1 contains e1

from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.test import convert_mem

m = '02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021'
c1 = cbd2_hash(m,4)
c2 = cbd2_hash(m,5)
c3 = cbd2_hash(m,6)

print(c1)
print(c2)
print(c3)

out11 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_0.txt")
out21 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_1.txt")
out31 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_2.txt")
out12 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_0.txt")
out22 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_1.txt")
out32 = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_2.txt")

check_dual(c1,out11,out12)
check_dual(c2,out21,out22)
check_dual(c3,out31,out32)

[1, 0, -1, 0, 1, -2, 0, 0, -1, -1, -1, -1, 0, 1, 1, -1, -1, -1, 1, 2, 0, 0, -1, 1, 0, 0, -1, 0, 1, -2, 0, -1, 0, 0, 1, -2, -1, 0, 0, -2, 1, -1, 2, 0, 1, -1, 0, 2, 0, 0, 0, 1, 0, 0, 1, -1, -1, 0, 2, 0, 2, 0, -1, 0, -1, 0, -1, -1, 1, -2, -1, 0, -1, 0, -2, 0, -1, 0, 0, 0, 0, 1, 0, 1, -1, 0, 0, 0, 0, -1, 1, -2, -1, 0, 0, 1, 1, -1, -1, 0, 0, 2, -1, 0, -1, -2, 0, 0, 1, 2, 0, 0, 1, 0, -1, 0, -1, -2, -1, -2, -1, 0, -1, 0, -1, -2, 0, 1, -1, -2, 1, 1, -1, 1, 0, 0, -1, -1, -1, 1, -2, -2, 1, -2, 1, -1, 1, 1, 0, -1, 0, 2, -1, -1, 0, 1, 1, 0, -1, 0, 0, 0, 0, -1, -1, 0, 1, -1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, -2, -1, 0, 0, -1, -2, 1, 1, 0, 1, -2, 0, 1, 1, -1, 0, -1, -1, 0, 0, 0, 2, -2, 1, -1, -1, 1, 1, 0, 1, 2, -1, -2, 0, -2, 0, 0, 0, 0, -1, 0, -2, 0, 0, 1, -1, 2, 0, -2, 0, 1, 0, 0, -1, -1, 1, 0, 0, -1, -1, 0, 1, 1, 1, 0, 1, -2, 0, 0, 0, 1, -1, 0, -1, 1, 0, -1, 0]
[0, -1, -1, 1, 0, -1, 1, -1, 0, 1, 1, -1, 1, -1, 1, 1, -1, 0, 1, 1, -1, 1, 0, -1, 2, 2, -1, -1, -1, 0, 1, 0, -1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 2

In [19]:
# checks c1 in ACCU1
# c1 = Encode(Compress(u)), u =InvNTT(A * y) + e1
# simulation time: 116685ps when ACCU1 contains C1 (and part of u)
from ipynb.fs.defs.matrix_hash import samplentt_suite
from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.ntt_invntt import invntt, barrett_reduce
from ipynb.fs.defs.checking import check_dual
from ipynb.fs.defs.encode import compress_encode_10
from ipynb.fs.defs.test import convert_mem
from ipynb.fs.defs.compress import compress
from ipynb.fs.defs.polyvec import polyvec_basemul_acc_mont, poly_add, poly_reduce

m_matrix ="0708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f20212223242526" 
m_cbd = "02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021"
AA = [[]]*3*3
Ay = [[]]*3
Intt_Ay = [[]]*3
accu1 = [[]]*3
e1 = [[]]*3
c1 = [[]]*3
mem_accu1 = [[]]*2*3

# matrix sample ntt
for i in range(3):
    for j in range(3):
        AA[i*3+j] = samplentt_suite(m_matrix,j,i)
        # print(i,j,AA[i][j])

# A * y
for i in range(3):
    Ay[i] = polyvec_basemul_acc_mont(AA[i*3+0], AA[i*3+1], AA[i*3+2], m_cbd,0,1,2)
    print("A*y:",i, Ay[i])
    Intt_Ay[i] = invntt(Ay[i])
    # print(i, Ay[i][::-1])
    e1[i] = cbd2_hash(m_cbd, i+4 )
    print("e1:",i,e1[i])
    print("InvNTT(A*y):",i,Intt_Ay[i])
    print()

for i in range(3):
    accu1[i] = (poly_add(e1[i], Intt_Ay[i]))
    accu1[i] = poly_reduce(accu1[i])

for i in range(3):
    c1[i] = compress_encode_10(accu1[i])

for i in range(3):
    mem_accu1[i*2 + 0] = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_a-k_"+str(i)+".txt")
    mem_accu1[i*2 + 1] = convert_mem("../sim_output/tb_kyber_pke_enc-accu1-ram_b-k_"+str(i)+".txt")
    # print(mem_accu1[i*2 + 0])
    # print(mem_accu1[i*2 + 1])

compressed_mem = [[]]*2*3
for i in range(3):
    for j in range(2):
        compressed_mem[i*2+j] = to_hex(mem_accu1[i*2 + j][0:80])
        print("c1:",i,"even or odd:",j,compressed_mem[i*2+j])

# test data produced by C code, this is compressed(u[0])
test = [445, 366, 360, 977,
892, 884, 765, 538,
105, 110, 982, 671,
835, 22, 505, 618,
107, 793, 657, 462,
30, 784, 188, 371,
231, 515, 504, 345,
542, 356, 635, 555,
54, 290, 100, 604,
233, 145, 612, 222,
15, 654, 501, 411,
459, 949, 420, 267,
400, 1008, 583, 385,
143, 175, 850, 370,
143, 506, 233, 260,
434, 683, 795, 689,
337, 530, 914, 113,
711, 747, 403, 554,
364, 142, 176, 731,
643, 832, 159, 592,
560, 217, 490, 57,
276, 2, 174, 769,
779, 542, 933, 369,
70, 746, 333, 541,
96, 665, 683, 861,
846, 41, 96, 54,
609, 30, 515, 906,
314, 939, 989, 436,
305, 14, 16, 491,
778, 933, 479, 822,
136, 703, 700, 918,
420, 864, 941, 500,
425, 35, 699, 305,
858, 410, 691, 1,
3, 329, 718, 31,
483, 126, 299, 268,
184, 876, 374, 599,
605, 865, 67, 862,
646, 445, 975, 265,
197, 32, 301, 544,
881, 416, 950, 0,
297, 186, 995, 879,
712, 997, 427, 261,
495, 116, 230, 415,
622, 81, 205, 849,
736, 1010, 435, 391,
621, 236, 62, 464,
578, 657, 970, 165,
494, 888, 628, 467,
883, 769, 883, 351,
107, 881, 419, 501,
843, 926, 439, 487,
755, 222, 110, 131,
258, 656, 160, 428,
619, 438, 87, 864,
807, 984, 635, 117,
832, 765, 820, 440,
212, 502, 1004, 663,
164, 94, 295, 149,
521, 99, 97, 948,
573, 743, 351, 544,
298, 772, 557, 400,
203, 777, 730, 654,
212, 205, 512, 513]

# checking it against my own result
for i in range(len(accu1[0])):
    t = compress(accu1[0][i],10,32)
    if(test[i]!=t):
        print("[+] proc")

for i in range(3):
    check_dual(c1[i],compressed_mem[i*2],compressed_mem[i*2+1])

print("[+] done")

A*y: 0 [879, 1623, 1340, 866, -222, -459, 1350, 1394, -796, -134, -728, -696, -1174, 401, -1367, -1104, -1484, -1168, 839, 1091, 727, 328, -459, -354, -1543, 1047, -83, -966, 458, 951, -358, 210, -1377, -437, -620, -1267, 273, 975, -318, 984, 469, 1539, 1025, -1184, 1279, -983, 1014, 928, -5, -1108, 1184, 1313, -1647, 1512, 916, 976, 933, -484, -810, -79, -1637, 180, -212, -923, -863, 1104, -1378, -410, -697, 534, 886, 1499, -604, 920, 808, 520, -1663, -1569, 857, 1575, -639, 1506, 1389, 542, 471, -611, 448, 265, -1248, -541, 917, 1609, 282, 521, -766, -393, 283, -624, -195, 984, -561, -209, -102, -761, 963, -943, 1134, -1518, -1572, -1527, 128, 734, -594, 1224, -574, -24, -320, 262, 739, -300, 1073, 63, 56, 961, 1482, 474, 568, -533, 548, -781, 398, -1223, -1531, 46, -792, 132, 1430, -1374, 959, -224, -1416, 1028, -1117, 1134, 380, 1394, 894, 399, -1385, -1352, -231, -355, -1573, -45, 257, -1417, 1075, 145, -345, 984, 514, 1187, 301, 253, -1317, -1133, -1108, -1567, 1307, 325, -1049, 

In [20]:
# checks c2 in ACCU2
# v = DECOMPRESS1(msg) + CBD(m,3) + InvNTT(t*y)
# t (vector) = encryption key t
# y (vector) = NTT(CBD(m,0)), NTT(CBD(m,1)), NTT(CBD(m,2))
# simulation time: 62465ps when ACCU2 contains c2 (and part of v)

from ipynb.fs.defs.decompress import decompress1_8_list
from ipynb.fs.defs.cbd2 import cbd2_hash
from ipynb.fs.defs.polyvec import polyvec_basemul_acc_mont, poly_add
from ipynb.fs.defs.ntt_invntt import invntt
from ipynb.fs.defs.test import convert_mem
from ipynb.fs.defs.encode import compress_encode_4

msg = [
 5,  6,  7,  8,  9, 10, 11, 12,
13, 14, 15, 16, 17, 18, 19, 20,
21, 22, 23, 24, 25, 26, 27, 28,
29, 30, 31, 32, 33, 34, 35, 36 ]
m_cbd = "02030405060708090a0b0c0d0e0f101112131415161718191a1b1c1d1e1f2021"
ekt = [[]]*3
for i in range(3):
    temp = []
    for j in range(256):
        temp.append(i*256 + j)
    ekt[i] = temp

# t*y
ty = polyvec_basemul_acc_mont(ekt[0],ekt[1],ekt[2],m_cbd,0,1,2)
Intt_ty = invntt(ty)
e2 = cbd2_hash(m_cbd,3)
mu = decompress1_8_list(msg)

v = poly_add(mu,e2)
v = poly_add(v,Intt_ty)

mem_accu2_a = convert_mem("../sim_output/tb_kyber_pke_enc-accu2-ram_a.txt")
mem_accu2_b = convert_mem("../sim_output/tb_kyber_pke_enc-accu2-ram_b.txt")

compressed_accu2_mem = [[]]*2
compressed_accu2_mem[0] = to_hex(mem_accu2_a[0:32])
compressed_accu2_mem[1] = to_hex(mem_accu2_b[0:32])

c2 = compress_encode_4(v)
print("c2:",c2)
check_dual(c2,compressed_accu2_mem[0],compressed_accu2_mem[1])

print("[+] done")


c2: ['0x26cb', '0x7827', '0x7d9d', '0x2dc4', '0x2cc5', '0xea58', '0xe955', '0xed57', '0x8d08', '0x80b7', '0xc40e', '0x2fdc', '0x4988', '0xf300', '0xa79a', '0x7ff8', '0x1ab0', '0xeafe', '0xf33e', '0x6d01', '0xa0b7', '0xe860', '0x92c5', '0xd7fa', '0xcc78', '0x7671', '0x2cc8', '0xf3b9', '0x1265', '0x36a0', '0xa009', '0xe621', '0xe4f5', '0xc369', '0x6641', '0x4e49', '0xa9f9', '0x9d20', '0xef34', '0x48d7', '0x6640', '0x4b34', '0xd114', '0x1315', '0x391e', '0xb91a', '0x730b', '0x4fa3', '0x1ed', '0xea2c', '0xe51a', '0xff7d', '0xc356', '0x663c', '0x5a60', '0xedbc', '0xb384', '0x1584', '0xbd9f', '0xa73e', '0xd64a', '0xfc26', '0x5b97', '0xd2a5']
[+] checking...
[+] done
